In [1]:
import requests
from IPython.core.display import display, HTML

#Dbpedia Spotlight

In [2]:

# An API Error Exception
class APIError(Exception):
  def __init__(self, status):
          self.status = status
  def __str__(self):
        return "APIError: status={}".format(self.status)

In [6]:
# Base URL for Spotlight API
base_url = "http://api.dbpedia-spotlight.org/en/annotate"

# Parameters # 'text' - text to be annotated # 'confidence' -   confidence score for linking
text ="""Albert Einstein was a German-born theoretical physicist who is best known for developing the theory of relativity. In 1905, he was awarded a PhD by the University of Zurich and received the 1921 Nobel Prize in Physics "for his services to theoretical physics.
"""
params = {"text": text, "confidence": 0.20}

 # Response content type
headers = {'accept': 'text/html'}


In [8]:
# Response content typeheaders = {'accept': 'text/html'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200 :
  raise APIError(res.status_code)# Something went wrong   

In [9]:
# Display the result as HTML in Jupyter Notebook
display(HTML(res.text))

#SpaCy NER model

In [10]:
!pip install -U spacy 

     |████████████████████████████████| 6.0 MB 5.4 MB/s 
     |████████████████████████████████| 181 kB 50.7 MB/s 
     |████████████████████████████████| 451 kB 51.8 MB/s 
     |████████████████████████████████| 10.1 MB 36.0 MB/s 
     |████████████████████████████████| 628 kB 47.0 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [11]:
!python -m spacy download en_core_web_sm 

     |████████████████████████████████| 13.9 MB 5.1 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [13]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Albert Einstein 0 15 PERSON
German 22 28 NORP
1905 118 122 DATE
PhD 141 144 WORK_OF_ART
the University of Zurich 148 172 ORG
1921 190 194 DATE
Nobel Prize 195 206 WORK_OF_ART


In [14]:
# display sentence involving original entities
spacy.displacy.render(doc,jupyter=True, style = "ent")

In [15]:
from spacy import displacy
displacy.render(next(doc.sents), style='dep', jupyter=True)


# Train your own NER model with SpaCy

In [16]:
!wget https://raw.githubusercontent.com/MaastrichtU-IDS/prodigy-drug-indication-annotation/master/relation/dailymed_disease3.jsonl

--2022-02-07 13:42:20--  https://raw.githubusercontent.com/MaastrichtU-IDS/prodigy-drug-indication-annotation/master/relation/dailymed_disease3.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2154026 (2.1M) [text/plain]
Saving to: ‘dailymed_disease3.jsonl’

dailymed_disease3.j 100%[===================>]   2.05M  --.-KB/s    in 0.06s   

2022-02-07 13:42:20 (33.8 MB/s) - ‘dailymed_disease3.jsonl’ saved [2154026/2154026]



In [17]:
import os
import json

In [18]:

training_data = []
with open('dailymed_disease3.jsonl') as fr:
    for count, line in enumerate(fr):
        #print (line)
        annotations = json.loads(line)
        
        text = annotations['text']
        #print (annotations['relations'])
        entity_mentions= []
        for a in annotations['spans']:
            
            if a['label'] == 'DISEASE':
                entity_mention = (a['start'], a['end'], 'DISEASE')
            #if entity_mention1 not in entity_mentions:
            entity_mentions.append(entity_mention)
                
        if len(entity_mentions) > 0:
            training_data.append( (text,  {'entities': entity_mentions}))   

In [19]:
len(training_data)

555

In [23]:
training_data[4]

('    1    [See     Clinical Studies (14.2)         Ciprofloxacin tablets are indicated for treatment of plague, including pneumonic and septicemic plague, due to     Yersinia pestis (Y. pestis)    [see          Clinical Studies (14.3)    ]     .',
 {'entities': [(103, 109, 'DISEASE'), (135, 152, 'DISEASE')]})

In [24]:
#import pandas as pd
from tqdm import tqdm
#import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm( training_data[:-55]): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        #print (start, end, label)
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
            print (start, end, label)
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("train.spacy") # save the docbin object

100%|██████████| 500/500 [00:00<00:00, 910.32it/s]


In [25]:
db = DocBin()
for text, annot in tqdm(training_data[-55:]): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
            print (start, end, label)
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("valid.spacy") # save the docbin object

100%|██████████| 55/55 [00:00<00:00, 1130.66it/s]


In [26]:
!pip install spacy-transformers

     |████████████████████████████████| 51 kB 129 kB/s 
     |████████████████████████████████| 3.4 MB 9.4 MB/s 
     |████████████████████████████████| 1.1 MB 46.5 MB/s 
     |████████████████████████████████| 895 kB 47.5 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
     |████████████████████████████████| 3.3 MB 22.9 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [27]:
# Go to this link "https://spacy.io/usage/training" and generate base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [28]:
!python -m spacy train config.cfg --verbose --output spacy-ner-model  --paths.train train.spacy --paths.dev valid.spacy  --gpu-id 0

[2022-02-07 13:51:55,423] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
✔ Created output directory: spacy-ner-model
ℹ Saving to output directory: spacy-ner-model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-02-07 13:52:04,925] [INFO] Set up nlp object from config
[2022-02-07 13:52:04,937] [DEBUG] Loading corpus from path: valid.spacy
[2022-02-07 13:52:04,939] [DEBUG] Loading corpus from path: train.spacy
[2022-02-07 13:52:04,939] [INFO] Pipeline: ['transformer', 'ner']
[2022-02-07 13:52:04,945] [INFO] Created vocabulary
[2022-02-07 13:52:04,946] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 780kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 5.15MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 3.14MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 7.61MB/s]
Downloading: 100% 478M/478M [00:11<00:00, 43.3MB/s]
Some weights of the model checkpoint at roberta-base were not used when init

In [1]:
import spacy

In [2]:
ner = spacy.load("spacy-ner-model/model-best") #load the best model

In [5]:
text = "Ciprofloxacin tablets are indicated in adult patients for treatment of bipolar disease and urinary tract infections  caused by  Escherichia coli "
spacy.displacy.render(ner(text),jupyter=True, style = "ent")